In [1]:
import graphviz
from graphviz import Digraph
import csv
import numpy as np
import util
from util import createPetri
from util import parsePlainFile
from util import remove_nodes
from util import add_nodes
from util import offsetEquation
import os
from PIL import Image
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

In [2]:
import nltk
nltk.download('words')
from nltk.corpus import words
word_list = words.words()

[nltk_data] Downloading package words to /Users/shivam/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
if not os.path.isdir('data'):
    os.mkdir('data')
    
if not os.path.isdir('descriptions'):
    os.mkdir('descriptions')

if not os.path.isdir('plain_data'):
    os.mkdir('plain_data')
    
if not os.path.isdir('edge_data'):
    os.mkdir('edge_data')
    
if not os.path.isdir('mask_data'):
    os.mkdir('mask_data')

In [4]:
maxNodes = 8
count = 0
NO_Samples = 50
for i in range(2, maxNodes):
    print (i)
    for j in range(NO_Samples):
        ranInts = np.random.randint(len(word_list), size=i)
        word_list = np.asarray(word_list)
        words = word_list[ranInts]
        createPetri(words, count)
        count += 1

2


KeyboardInterrupt: 